# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#imports 
import pandas as pd
import numpy as np

from scipy import stats
import seaborn as sna
import matplotlib as plt
%matplotlib inline

In [3]:
#explore the data 
df = pd.read_csv('homepage_actions.csv')
df.head()


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [6]:
df['id'].value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [11]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [25]:
#How many viewers also clicked? Above we see 1,860 both clicked and viewed. Whats the rate of both clicking and viewing?

df_click = df.query('action =="click"')
df_view = df.query('action == "view"')
df_view['action'].count()

just_view = df_view['action'].count()- df_click['action'].count()
click_and_view = df_click['action'].count()

click_through_rate = click_and_view/just_view
click_through_rate

# according to our data, 41 percent of viewers also clicked 

0.41629364368845123

In [35]:
cids = set(df[df.action=='click']['id'].unique())
vids = set(df[df.action=='view']['id'].unique())
print("Number of viewers: {} \tNumber |of clickers: {}".format(len(vids), len(cids)))
print("Number of Viewers who didn't click: {}".format(len(vids-cids)))
print("Number of Clickers who didn't view: {}".format(len(cids-vids)))

Number of viewers: 6328 	Number |of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


In [38]:
#Is there any overlap between the control and experiment groups? 
exp = set(df[df.group=='experiment']['id'].unique())
cont= set(df[df.group=='control']['id'].unique())
print("Number of individials who were in both the experimental and control groups: {}".format(len(exp&cont)))

#Nope! No overlap, so we are good to go! 

Number of individials who were in both the experimental and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [46]:
#make two groups
df['count']= 1
df_exp = df[df.group=='experiment']
df_cont = df[df.group=='control']
#check variences 
print("Number of experimental subjects: {} \tNumber of control subjects: {}".format(len(df_exp), len(df_cont)))
print("Experimental varience: {} \tControl varience: {}".format(np.var(df_exp), np.var(df_cont)))
""" Here we see not such a huge difference between the varience (less than 4:1) but there is a possibly signifigant 
difference in thier sample sizes, so just to be safe lets use Welch's T-test."""

Number of experimental subjects: 3924 	Number of control subjects: 4264
Experimental varience: id       4.782388e+10
count    0.000000e+00
dtype: float64 	Control varience: id       4.814075e+10
count    0.000000e+00
dtype: float64


" Here we see not such a huge difference between the varience (less than 4:1) but there is a possibly signifigant \ndifference in thier sample sizes, so just to be safe lets use Welch's T-test."

In [47]:
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [53]:
# reorganize the data
control = df_cont.pivot(index='id', columns='action', values='count')
print(control.head())
""" We see lots of NaN's here - let's change those to 0 so we can do caluculations with our data"""


action  click  view
id                 
182994    1.0   1.0
183089    NaN   1.0
183248    1.0   1.0
183515    NaN   1.0
183524    NaN   1.0


" We see lots of NaN's here - let's change those to 0 so we can do caluculations with our data"

In [54]:
control = control.fillna(0)

In [56]:
#same thing with the experimental group 
experiment = df_exp.pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(0)

In [57]:
#looking at the information we've organized

print("Sample sizes: \tControl:{}, Experiment:{}".format(len(control), len(experiment)))
print("Total clicks: \tControl:{}, Experiment:{}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate: \tControl:{}, Experiment:{}".format(control.click.mean(), experiment.click.mean()))

Sample sizes: 	Control:3332, Experiment:2996
Total clicks: 	Control:932.0, Experiment:928.0
Average click rate: 	Control:0.2797118847539016, Experiment:0.3097463284379172


In [59]:
# we see that there is an increase in clicks between the control
#and experimental groups, but is it signifigant? 

In [60]:
print(stats.ttest_ind(control['click'], experiment['click'], equal_var = False))

Ttest_indResult(statistic=-2.615440020788211, pvalue=0.008932805628674201)


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [62]:
#find click through rate of the control group for the expected
control_rate = control['click'].mean()
expected_experiment_clicks = control_rate*len(experiment)
print(expected_experiment_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [65]:
#Your code here
n = len(experiment)
p= control_rate
var = n*p*(1-p)
std = np.sqrt(var)
print(std)


24.568547907005815


In [68]:
observed_experiment_clicks = experiment["click"].sum()
z_score = (observed_experiment_clicks -expected_experiment_clicks)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [69]:
stats.norm.sf(abs(3.6625360854823588))

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
# it's close enough, and both would reject H0 with an alpha of 0.05 
#(or even a more conservative alpha of 0.01)

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.